In [14]:
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install -r requirements.txt
!pip install .

Cloning into 'KoBERT'...
remote: Enumerating objects: 428, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 428 (delta 131), reused 110 (delta 108), pack-reused 273
Receiving objects: 100% (428/428), 221.30 KiB | 7.14 MiB/s, done.
Resolving deltas: 100% (219/219), done.
/content/KoBERT/KoBERT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/KoBERT/KoBERT
  Preparing metadata (setup.py) ... done
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15708 sha256=57aea3865d8d32c4639da23a5b3268110517bfd63c7848d2cb9f88d4a471ba83
  Stored in directory: /tmp/pip-ephem-wheel-cache-jbmxr9ku/wheels/bc/07/0d/328e685154d77a70ff49d5492fcb8c335559affb636f479989
Successfully built kobert
  Attempting uninstall: kobert
    Found existing installation:

In [2]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers
# !pip install torch

In [15]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [16]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [17]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [18]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-7tv2w27m/kobert-tokenizer_fbd4ca2f291c45d0b708bdd7c82bf74d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-7tv2w27m/kobert-tokenizer_fbd4ca2f291c45d0b708bdd7c82bf74d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=1ede2f8938cdc32f0a4b420e26f4694541487a0c73b94a1076e7cffb9c1d9243
  Stored in directory: /tmp/pip-ephem-wheel-cache-xdykvuek/wheels/f7/cb/29/1a737fe71e5108dc30b04ea4a990f78ed271fa537aaf3fce7c
Successfully built kobert_tokenizer


In [19]:
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model_path = 'skt/kobert-base-v1'
vocab_file = tokenizer.vocab_file
bertmodel, vocab = get_pytorch_kobert_model()

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

/content/KoBERT/KoBERT/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KoBERT/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [20]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

# Saved 모델 불러오는 코드

In [8]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3.0.2
# !pip install torch

# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

# torch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기 필수
bertmodel, vocab = get_pytorch_kobert_model()


# KoBERT에 입력될 데이터셋 정리
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

# 모델 정의
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

## 학습 모델 로드
device = torch.device('cpu')
PATH = '/content/drive/MyDrive/Colab_Notebooks/music_resting_place/saved_model/' # 경로 수정 필요
model = torch.load(PATH + 'saved_modelKoBERT_.pt', map_location=device)  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'saved_modelmodel_state_dict.pt', map_location=device))  # state_dict를 불러 온 후, 모델에 저장

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def new_softmax(a) : 
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)


# 예측 모델 설정
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long()#.to(device)
        segment_ids = segment_ids.long()#.to(device)

        valid_length= valid_length
        label = label.long()#.to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            probability = []
            logits = np.round(new_softmax(logits), 3).tolist()
            for logit in logits:
                print(logit)
                probability.append(np.round(logit, 3))

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("분노가")
            elif np.argmax(logits) == 2:
                test_eval.append("불안이")
            elif np.argmax(logits) == 3:
                test_eval.append("상처가")
            elif np.argmax(logits) == 4:
                test_eval.append("슬픔이")

            
            print(probability)
            print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
    return probability

In [9]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 아 나는 왜 이렇게 잘하는게 없을까?
0.4749999940395355
63.229000091552734
16.033000946044922
5.705999851226807
14.557000160217285
[0.475, 63.229, 16.033, 5.706, 14.557]
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 나 오늘 기분이 너무 좋아
99.60800170898438
0.07800000160932541
0.13099999725818634
0.09399999678134918
0.08900000154972076
[99.608, 0.078, 0.131, 0.094, 0.089]
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 상욱이가 너무 무서워
0.42800000309944153
42.57600021362305
30.924999237060547
23.975000381469727
2.0959999561309814
[0.428, 42.576, 30.925, 23.975, 2.096]
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 병창이는 천재야
97.68499755859375
0.5899999737739563
0.6029999852180481
0.8769999742507935
0.24500000476837158
[97.685, 0.59, 0.603, 0.877, 0.245]
>> 입력하신 내용에서 기쁨이 느껴집니다.


하고싶은 말을 입력해주세요 : 우리 팀이 너무 고생이 많아서 걱정이야
3.4690001010894775
11.026000022888184
33.051998138427734
40.558998107910156
11.892999649047852
[3.469, 11.026, 33.052, 40.559, 11.893]
>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 머신러닝, 딥러닝은 너무 어려워. 잘 

# 무시해도 되는 코드

In [ ]:
# import torch
# import torch.nn as nn
# import random

# from model.classifier import KoBERTforSequenceClassfication
# from kobert_transformers import get_tokenizer


# def load_wellness_answer():
#     root_path = "."
#     category_path = f"{root_path}/data/wellness_dialog_category.txt"
#     answer_path = f"{root_path}/data/wellness_dialog_answer.txt"

#     c_f = open(category_path, 'r')
#     a_f = open(answer_path, 'r')

#     category_lines = c_f.readlines()
#     answer_lines = a_f.readlines()

#     category = {}
#     answer = {}
#     for line_num, line_data in enumerate(category_lines):
#         data = line_data.split('    ')
#         category[data[1][:-1]] = data[0]

#     for line_num, line_data in enumerate(answer_lines):
#         data = line_data.split('    ')
#         keys = answer.keys()
#         if (data[0] in keys):
#             answer[data[0]] += [data[1][:-1]]
#         else:
#             answer[data[0]] = [data[1][:-1]]

#     return category, answer


# def kobert_input(tokenizer, str, device=None, max_seq_len=512):
#     index_of_words = tokenizer.encode(str)
#     token_type_ids = [0] * len(index_of_words)
#     attention_mask = [1] * len(index_of_words)

#     # Padding Length
#     padding_length = max_seq_len - len(index_of_words)

#     # Zero Padding
#     index_of_words += [0] * padding_length
#     token_type_ids += [0] * padding_length
#     attention_mask += [0] * padding_length

#     data = {
#         'input_ids': torch.tensor([index_of_words]).to(device),
#         'token_type_ids': torch.tensor([token_type_ids]).to(device),
#         'attention_mask': torch.tensor([attention_mask]).to(device),
#     }
#     return data


# if __name__ == "__main__":
#     root_path = "."
#     checkpoint_path = f"{root_path}/checkpoint"
#     save_ckpt_path = f"{checkpoint_path}/kobert-wellness-text-classification.pth"

#     # 답변과 카테고리 불러오기
#     category, answer = load_wellness_answer()

#     ctx = "cuda" if torch.cuda.is_available() else "cpu"
#     device = torch.device(ctx)

#     # 저장한 Checkpoint 불러오기
#     checkpoint = torch.load(save_ckpt_path, map_location=device)

#     model = KoBERTforSequenceClassfication()
#     model.load_state_dict(checkpoint['model_state_dict'])

#     model.to(ctx)
#     model.eval()

#     tokenizer = get_tokenizer()

#     while 1:
#         sent = input('\nQuestion: ')  # '요즘 기분이 우울한 느낌이에요'
#         data = kobert_input(tokenizer, sent, device, 512)

#         if '종료' in sent:
#             break

#         output = model(**data)

#         logit = output[0]
#         softmax_logit = torch.softmax(logit, dim=-1)
#         softmax_logit = softmax_logit.squeeze()

#         max_index = torch.argmax(softmax_logit).item()
#         max_index_value = softmax_logit[torch.argmax(softmax_logit)].item()

#         answer_list = answer[category[str(max_index)]]
#         answer_len = len(answer_list) - 1
#         answer_index = random.randint(0, answer_len)
#         print(f'Answer: {answer_list[answer_index]}, index: {max_index}, softmax_value: {max_index_value}')
#         print('-' * 50)